In [ ]:
import os
#from google.colab import drive
#drive.mount('/content/drive')
#os.chdir('/content/drive/MyDrive/Github/dus_mm/')
import numpy as np
import pandas as pd
import utils
import glob

from matplotlib import pyplot as plt
import seaborn as sb
os.chdir('data2')

columnsX = ['freq','AX','BX','BY','CY','CX','DX','DY','AY']
columnsY = ['AX','BX','BY','CY','CX','DX','DY','AY','AQ','AL','BQ','BL','CQ','CL','DQ','DL']
columnsYout = ['AQ','AL','BQ','BL','CQ','CL','DQ','DL','X','Y','Q','L']

FileNotFoundError: [WinError 2] Не удается найти указанный файл: 'data2'

In [ ]:
X = pd.DataFrame()
y = pd.DataFrame()
numb_files = len(glob.glob('data_output*'))
for i in glob.glob('data_output*'): 
    numb = i[11]
    if (numb == 'data_output.txt'):
        numb = ''
    y = pd.concat([y,pd.read_table(os.path.join('data_output%s.txt' % numb), header = 0, names = columnsY, 
                                   index_col = False, sep='\s+', engine='python', dtype ='float64')], ignore_index = 1)
    X = pd.concat([X,pd.read_table(os.path.join('data_input%s.txt' % numb), header = 0, names = columnsX, 
                                   index_col = False, sep='\s+', engine='python', dtype ='float64')], ignore_index = 1) 
nelem = 64
freq_bias = [252.92, 97.13, 87.06, 57.57, 12.55, 3.40, -4.78, -11.92]
#freq_bias = [9.8,   4.9,   1.03, -12.05]
#freq_bias = [0, 0, 0, 0, 0, 0, 0, 0]
X = utils.init_in(X, freq_bias)
y = utils.decomposition(y)

In [ ]:
alfa = 1
koef = np.zeros(len(freq_bias))
X1 = utils.init_in(X.copy(), freq_bias)
for i in range (100):
    for n in range(len(freq_bias)):
        koef[n] = (0.5 - X1['freq' + str(n+1)].mean())
    freq_bias = freq_bias + koef * alfa
    X1 = utils.init_in(X.copy(), freq_bias)

In [ ]:
class Layer():
    def __init__(self, neurons: int = 256, activation: str = 'linear'):
        self.neurons = neurons
        self.activation = activation

In [ ]:
class NN_model():
  
    def __init__(self, input_dimension: int, loss = 'mean_squared_error'):
        self.sequential = [Layer(input_dimension)]
        self.theta = np.array([])
        self.loss = loss
        self.optimizer = 'adam' #'adam'
        self.max_epoch = 1000
        self.chunk_size = 32
        self.beta_1 = 0.9
        self.beta_2 = 0.99
        self.epsilon = 1e-8
        self.learning_rate = 0.01
        self.regularization = None #'l2'
        self.best_loss_val = 1e10

    def add(self, layer: Layer):
        self.sequential.append(layer)
        layer_in = self.sequential[-2]
        layer_out = self.sequential[-1]
        gamma = np.sqrt(6)/np.sqrt(layer_in.neurons + layer_out.neurons)
        np.random.seed(0)
        init_theta = np.random.rand(layer_out.neurons,layer_in.neurons + 1) * 2 * gamma - gamma
        self.theta = np.hstack([self.theta, init_theta.ravel()])


    def activation(self, layer: Layer, x, diff: bool = False):
        if (layer.activation == 'relu'):
            if (diff == True):
                x[x >= 0] = 1
                x[x < 0] = 0
            else:
                x[x < 0] = 0
        elif (layer.activation == 'linear'):
            if (diff == True):
                x = 1
            else:
                x = x
        elif (layer.activation == 'sigmoid'):
            if (diff == True):
                x = self.activation(layer, x) * (1 - self.activation(layer, x))
            else:
                x = 1/(1 + np.exp(-x))
        return x

    def feedforward(self, x, theta_new = None):
        if (theta_new is None):
            theta_new = self.theta
        a = x
        a_out = np.array(a.ravel())
        slice_theta1 = 0
        for i in range(1, len(self.sequential)):
            layer1 = self.sequential[i - 1]
            layer2 = self.sequential[i]
            slice_theta2 = slice_theta1 + (layer1.neurons + 1) * layer2.neurons
            
            theta1 = theta_new[slice_theta1:slice_theta2].reshape(layer2.neurons, layer1.neurons + 1)
            z = np.dot(np.concatenate((np.ones([a.shape[0],1]), a), axis = 1), theta1.T)
            a = self.activation(layer2, z)
            a_out = np.hstack([a.ravel(), a_out])
            slice_theta1 = slice_theta2
        return a_out

    def backprop(self, a_out, y, theta_new = None):
        if (theta_new is None):
            theta_new = self.theta
        grad = np.array([])
        slice_theta1 = 0
        slice_a1 = 0
        for i in range(1, len(self.sequential)):
            layer1 = self.sequential[-i]
            layer2 = self.sequential[-(i + 1)]
            slice_theta2 = slice_theta1 + layer1.neurons * (layer2.neurons + 1)
            slice_a2 = slice_a1 + layer1.neurons * y.shape[0]
            slice_a3 = slice_a2 + layer2.neurons * y.shape[0]           
            a1 = a_out[slice_a1:slice_a2].reshape((y.shape[0], layer1.neurons))
            a2 = a_out[slice_a2:slice_a3].reshape((y.shape[0], layer2.neurons))
            a2 = np.concatenate((np.ones([a2.shape[0], 1]), a2), axis = 1)
            theta2 = np.flip(theta_new)[slice_theta1:slice_theta2].reshape(layer1.neurons, layer2.neurons + 1)
            theta2 = np.flip(theta2)
            if (i == 1):
                delta = self.compute_loss(a_out, y, diff = True) * self.activation(layer1, np.dot(a2, theta2.T), diff = True)
            else:
                delta = np.dot(delta, theta1[:,1:]) * self.activation(layer1, np.dot(a2, theta2.T), diff = True)  
            grad = np.hstack([np.dot(delta.T, a2).ravel(), grad])
            
            theta1 = theta2  
            slice_theta1 = slice_theta2
            slice_a1 = slice_a2
        grad = (1 / y.shape[0]) * grad
        return grad

    def nulling(self, theta_new2 = None):
        if (theta_new2 is None):
            theta_new2 = self.theta.copy()
        slice_theta1 = 0
        for i in range(1, len(self.sequential)):
            layer1 = self.sequential[i - 1]
            layer2 = self.sequential[i]
            slice_theta2 = slice_theta1 + (layer1.neurons + 1) * layer2.neurons
            theta_new2[slice_theta1:slice_theta2:layer1.neurons] = 0
            slice_theta1 = slice_theta2
        return theta_new2
          

    def compute_loss(self, a_out, y, theta_new = None,  diff: bool = False):
        if (theta_new is None):
            theta_new = self.theta.copy()
        a_end = a_out[:(self.sequential[-1].neurons * y.shape[0])]
        a_end = a_end.reshape((y.shape[0], self.sequential[-1].neurons))
        if (self.loss == 'mean_squared_error'):
            if (diff == True):
                loss = (a_end - y)
            else:
                loss = np.sum(np.power((a_end - y), 2)) / (2 * y.shape[0])
        elif (self.loss == 'loglos'):
            if (diff == True):
                loss = (a_end - y)/((1 - a_end) * a_end)
            else:
                loss = - np.sum(y * np.log(a_end) + (1 - y) * np.log(1 - a_end))/y.shape[0]     
        return loss
    
    def form_output(self, X, y):
        a_out = self.feedforward(X_train)
        a_end = a_out[:(self.sequential[-1].neurons * y.shape[0])]
        return a_end

    def gradient_checking(self, x, y, e = 1e-4):
        numgrad = np.zeros(self.theta.shape)
        perturb = np.diag(e * np.ones(self.theta.shape))
        for i in range(self.theta.size):
            theta_plus = self.theta + perturb[:,i]
            theta_minus = self.theta - perturb[:,i]
            a_out_plus  = self.feedforward(x, theta_plus)
            a_out_minus = self.feedforward(x, theta_minus)
            numgrad[i] = (self.compute_loss(a_out_plus, y, theta_plus) - self.compute_loss(a_out_minus, y, theta_minus))/(2 * e)
        a_out = self.feedforward(x)
        grad = self.backprop(a_out, y)
        diff = np.linalg.norm(numgrad - grad)/np.linalg.norm(numgrad + grad)
        print('Relative Difference: %g' % diff)


    def optimize(self, X_train, y_train, X_val, y_val, lambda_new = 0 , theta_new = None):
        if (theta_new is None):
            theta_new = self.theta
        for epoch in range(self.max_epoch):
            m = np.zeros(self.theta.shape)
            v = np.zeros(self.theta.shape)
            lst =  np.arange(y_train.shape[0])
            np.random.shuffle(lst)
            chunks = np.array([lst[i: i + self.chunk_size] for i in range(0, y_train.shape[0], self.chunk_size)])
            for t, chunk in enumerate(chunks):
                xi = X_train[chunk]
                yi = y_train[chunk]
                a_out = self.feedforward(xi, theta_new)
                grad = self.backprop(a_out, yi, theta_new)  
                if (self.optimizer == 'adam'):
                    #Ti = 0.1 *
                    #self.lambda_ = 0.025 * np.sqrt(self.chunk_size/(y.shape[0]*
                    grad = grad + (lambda_new) * self.nulling(theta_new)
                    m = self.beta_1 * m + (1 - self.beta_1) * grad
                    v = self.beta_2 * v + (1 - self.beta_2) * np.power(grad, 2)
                    m_hat = m / (1 - np.power(self.beta_1, t + 1))
                    v_hat = v / (1 - np.power(self.beta_2, t + 1))
                    theta_new = theta_new - self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon) #- (lambda_new) * self.nulling(theta_new)
                elif (self.optimizer == 'gradient_descent'):
                    theta_new = theta_new - self.learning_rate * grad -  lambda_new * self.nulling(theta_new)
            a_out_train = self.feedforward(X_train, theta_new)
  
            loss_train = self.compute_loss(a_out_train, y_train, theta_new)# + lambda_new * np.sum(np.power(self.nulling(theta_new), 2))/2
            a_out_val = self.feedforward(X_val, theta_new)
            loss_val = self.compute_loss(a_out_val, y_val, theta_new) #+ lambda_new * np.sum(np.power(self.nulling(theta_new), 2))/2
            
            if (self.best_loss_val > loss_val):  
                self.theta = theta_new
                self.best_loss_val = loss_val        
                
            print(f'{np.round(loss_train,2)}    {np.round(loss_val,2)}    {epoch}')
        return loss_train, loss_val
    
    def __repr__(self):
        return str(len(self.sequential))

In [ ]:
dus_mm = NN_model(X.shape[1], 'loglos')
dus_mm.add(Layer(128, 'sigmoid'))
dus_mm.add(Layer(128, 'sigmoid'))
dus_mm.add(Layer(y.shape[1], 'sigmoid'))

In [ ]:
import scipy.io
mat = scipy.io.loadmat('mnist.mat')
X = mat['X']
y = mat['y']
y[y == 10] = 0
y0 = np.zeros((y.size, 10))
for i in range(y.shape[0]):
    y0[i,y[i]] = 1
y = y0

In [ ]:
dus_mm = NN_model(X.shape[1])
dus_mm.add(Layer(128, 'relu'))
dus_mm.add(Layer(128, 'relu'))
dus_mm.add(Layer(y.shape[1], 'linear'))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# накидываем тысячу точек от -3 до 3
X = np.linspace(-3, 3, 1000).reshape(-1, 1)

# задаём линейную функцию, которую попробуем приблизить нашей нейронной сетью
def f(x):    
    return 2 * np.sin(x) + 5

f = np.vectorize(f)

# вычисляем вектор значений функции
y = f(X)

# отрисовываем результат приближения нейросетью поверх исходной функции
plt.scatter(X_train, y, color='black', antialiased=True)
#plt.plot(, model.predict(x), color='magenta', linewidth=2, antialiased=True)
plt.show()

# выводим веса на экран
# for layer in model.layers:
#     weights = layer.get_weights()
#     print(weights)

In [ ]:
dus_mm = NN_model(X.shape[1], 'mean_squared_error')
dus_mm.add(Layer(y.shape[1], 'linear'))

In [ ]:
X = np.array(X)
y = np.array(y)
lambda_vec = [0]

nfold = 5
slice_fold = int(y.shape[0] // nfold)
lst =  np.arange(y.shape[:(5 * slice_fold)][0])
np.random.shuffle(lst)
list_fold = np.array([lst[i * slice_fold:(i + 1) * slice_fold] for i in range(nfold)])

train_error = np.zeros(nfold)
val_error = np.zeros(nfold)
train_reg = np.zeros(len(lambda_vec))
val_reg = np.zeros(len(lambda_vec))
theta_new = dus_mm.theta.copy()

for k in range(len(lambda_vec)):
    theta_new
    for i in range(nfold):
        X_train = X[np.delete(list_fold, i, 0).ravel()]
        y_train = y[np.delete(list_fold, i, 0).ravel()]
        X_val = X[list_fold[i]]
        y_val = y[list_fold[i]]
        train_error[i], val_error[i] = dus_mm.optimize(X_train, y_train, X_val, y_val, lambda_vec[k], theta_new)
    train_reg[k] = np.mean(train_error)  
    val_reg[k] = np.mean(val_error)

<ipython-input-24-5303f59573c3>:151: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  chunks = np.array([lst[i: i + self.chunk_size] for i in range(0, y_train.shape[0], self.chunk_size)])


2788.12    2635.9    0
2256.66    2146.25    1
1998.26    1907.79    2
1865.06    1737.22    3
1740.81    1665.9    4
1681.54    1607.2    5
1618.97    1547.9    6
1528.01    1523.51    7
1454.66    1443.75    8
1420.73    1409.6    9
1344.83    1329.96    10
1274.04    1285.84    11
1125.38    1169.49    12
1029.94    1125.43    13
879.28    1027.03    14
828.07    987.96    15
714.19    851.49    16
623.38    786.23    17
586.46    734.98    18
568.66    714.63    19
547.62    682.47    20
504.95    644.28    21
492.58    639.36    22
449.19    632.61    23
416.41    577.96    24
397.98    554.34    25
399.2    543.46    26
368.36    536.66    27
371.83    537.78    28
343.82    520.46    29
335.94    498.91    30
292.11    453.84    31
301.17    462.95    32
304.61    476.0    33
286.51    458.34    34
273.41    450.84    35
278.57    447.99    36
283.86    452.79    37
268.94    443.18    38
285.17    462.79    39
282.5    456.52    40
277.05    434.79    41
247.6    426.4    42
23

50.46    298.51    357
56.87    308.41    358
43.76    297.2    359
47.34    302.26    360
50.21    299.15    361
48.56    300.14    362
47.56    298.87    363
49.76    300.54    364
52.29    310.79    365
49.95    310.4    366
53.79    311.7    367
53.39    306.72    368
43.67    299.51    369
52.31    311.52    370
44.56    306.96    371
47.91    294.24    372
48.75    297.11    373
47.39    299.81    374
44.92    297.8    375
51.89    297.65    376
47.6    305.81    377
47.78    295.6    378
48.36    302.15    379
47.96    301.56    380
47.5    311.33    381
47.5    294.1    382
49.35    306.81    383
49.28    302.77    384
47.58    305.26    385
54.78    307.61    386
46.15    300.95    387
52.13    304.42    388
46.89    291.95    389
46.23    301.75    390
42.27    297.66    391
48.68    308.94    392
45.95    300.12    393
43.15    308.01    394
48.66    300.34    395
52.17    302.93    396
51.31    299.83    397
46.15    302.45    398
43.92    309.69    399
52.34    314.56    4

38.82    294.97    715
34.32    301.68    716
41.64    296.67    717
33.62    292.24    718
36.04    292.31    719
35.36    303.35    720
34.95    300.45    721
33.98    290.8    722
38.93    298.74    723
38.55    307.35    724
33.11    291.31    725
33.97    290.67    726
32.01    292.92    727
37.19    298.89    728
34.53    303.8    729
30.15    291.52    730
35.24    294.23    731
37.17    296.79    732
35.75    294.04    733
30.8    296.53    734
32.2    296.47    735
30.82    293.65    736
32.27    289.42    737
35.95    305.59    738
35.77    293.58    739
35.18    298.53    740
33.14    293.14    741
35.86    299.92    742
33.9    301.76    743
35.56    301.1    744
32.26    302.78    745
32.38    305.52    746
33.82    293.44    747
33.34    299.38    748
33.17    302.52    749
39.75    303.55    750
32.9    298.0    751
32.31    291.33    752
40.37    307.52    753
31.0    291.1    754
33.52    298.81    755
33.63    298.88    756
31.01    299.75    757
33.86    299.39    75

173.12    347.52    75
157.51    330.17    76
170.27    354.76    77
170.14    353.79    78
152.45    335.34    79
156.57    337.91    80
158.65    347.02    81
144.24    333.51    82
143.42    316.15    83
155.22    339.42    84
139.65    327.37    85
138.82    324.46    86
155.31    350.95    87
145.22    329.55    88
152.31    344.48    89
137.18    324.11    90
136.72    323.87    91
165.46    355.66    92
150.12    337.5    93
137.26    328.98    94
140.66    340.16    95
136.51    330.17    96
136.33    335.65    97
128.64    319.95    98
123.33    312.29    99
132.75    321.11    100
134.02    331.23    101
126.32    316.56    102
126.39    320.18    103
126.73    320.07    104
117.78    307.64    105
123.2    318.82    106
120.33    321.46    107
123.74    320.58    108
129.3    320.71    109
106.71    305.96    110
118.2    323.31    111
131.34    331.52    112
117.92    311.8    113
113.26    300.21    114
108.1    295.61    115
113.63    306.74    116
109.28    307.82    117

...

90.47    305.29    152
106.8    310.46    153
95.25    310.12    154
80.85    295.24    155
79.63    302.36    156
80.55    293.61    157
79.66    293.38    158
83.78    295.91    159
68.98    278.12    160
81.27    293.18    161
88.16    308.68    162
85.0    293.67    163
84.42    289.48    164
77.05    281.99    165
77.86    285.42    166
76.06    286.01    167
80.85    299.35    168
78.03    284.28    169
71.99    281.29    170
80.73    290.83    171
75.2    285.25    172
72.43    284.82    173
83.13    288.2    174
74.57    279.13    175
78.61    294.31    176
68.47    278.79    177
74.45    289.96    178
70.63    287.93    179
71.62    295.95    180
80.58    289.99    181
84.28    300.53    182
79.44    295.97    183
109.16    325.03    184
71.31    289.94    185
84.2    306.58    186
71.19    280.02    187
73.39    292.72    188
75.31    289.91    189
64.65    281.54    190
77.76    295.75    191
73.08    297.67    192
83.47    305.69    193
74.69    289.0    194
67.49    285.52

34.91    288.1    512
37.22    294.85    513
36.36    296.48    514
41.58    295.49    515
37.71    293.38    516
35.26    291.88    517
40.92    300.62    518
33.17    292.98    519
36.22    288.45    520
36.2    294.15    521
35.3    291.21    522
40.64    298.76    523
45.16    297.68    524
35.59    286.84    525
33.56    291.16    526
38.99    294.77    527
36.88    293.36    528
34.86    289.45    529
36.68    295.13    530
40.95    296.7    531
35.46    295.64    532
34.97    293.85    533
40.7    303.76    534
35.21    292.17    535
36.53    298.43    536
35.1    285.91    537
36.01    295.64    538
34.45    291.78    539
36.09    296.18    540
35.33    291.98    541
37.2    300.09    542
34.7    285.74    543
38.72    300.07    544
37.66    289.85    545
40.81    305.31    546
36.79    295.58    547
35.07    295.26    548
33.72    293.47    549
35.23    295.14    550
36.89    289.79    551
36.15    285.25    552
34.52    289.21    553
32.05    289.75    554
36.9    298.45    5

29.36    298.93    872
30.76    298.29    873
27.08    296.47    874
29.82    300.03    875
28.73    294.68    876
28.55    290.28    877
31.74    305.29    878
27.03    298.23    879
29.88    294.16    880
30.61    301.94    881
30.63    302.34    882
33.09    303.66    883
29.49    294.49    884
27.07    290.84    885
27.94    288.25    886
32.96    306.32    887
27.84    297.74    888
31.06    297.84    889
32.23    295.6    890
28.38    300.45    891
27.49    292.2    892
27.66    302.19    893
30.33    305.76    894
26.87    299.45    895
23.94    293.07    896
30.0    301.16    897
26.37    298.51    898
27.36    299.2    899
30.75    300.6    900
26.68    296.77    901
28.5    297.05    902
26.28    291.89    903
28.16    298.97    904
27.69    298.28    905
27.04    299.65    906
25.56    292.47    907
29.83    306.27    908
29.57    297.58    909
31.8    304.24    910
28.79    305.11    911
28.77    295.74    912
24.68    290.4    913
32.8    300.05    914
29.31    298.09    9

60.1    264.99    233
53.56    256.77    234
55.39    266.72    235
58.58    266.68    236
56.58    266.5    237
52.1    260.97    238
57.74    268.56    239
54.19    268.72    240
59.37    259.52    241
60.46    268.2    242
57.52    265.73    243
57.33    274.35    244
60.51    270.21    245
55.27    263.04    246
51.64    256.9    247
51.35    257.76    248
59.54    275.18    249
52.38    263.9    250
51.32    259.98    251
58.03    273.2    252
53.37    266.05    253
53.76    271.1    254
59.43    266.16    255
56.93    261.24    256
58.35    275.0    257
56.08    264.84    258
59.36    267.02    259
57.57    262.94    260
57.69    267.43    261
56.52    259.29    262
65.35    273.5    263
54.62    266.38    264
56.6    270.77    265
50.44    267.36    266
57.25    271.58    267
60.18    276.04    268
52.4    258.43    269
53.09    266.64    270
53.66    269.27    271
56.46    271.22    272
59.15    277.15    273
50.11    256.9    274
58.46    265.42    275
63.74    274.48    276
5

KeyboardInterrupt: 

['AQ','AL','BQ','BL','CQ','CL','DQ','DL','X','Y','Q','L']
  -1., -0., -2.,  1., 34., 65.,  5., 23., 81., -2., 18., 21
   2.,  1.,  1.,  0., 15., 65., 24.,  2., 83.,  1.,  7.,  4.
   0.,  1., -1.,  10.,21., 63., 26.,  6., 86.,  -1., 12.,-20.
      
   1.,  1.,  3.,  7., 47., 15., 20.,  48., 54.,  3.,-11., 60.

In [ ]:
XX = pd.read_table(os.path.join('data7A-1_2.txt'), header = 0, names = columnsX, index_col = False, sep='\s+', engine='python', dtype ="float")
XX = utils.init_in(XX,freq_bias)
yy = np.round(dus_mm.feedforward(np.array(XX))[:12])
#yy = np.abs(pd.DataFrame(yy.astype('float'), columns = columnsYout))
#yy = utils.integration(yy,nelem)
# predictions = NN_model.predict(XX)
# yy = np.abs(pd.DataFrame(predictions.astype('float'), columns = columnsYout))
#yy = utils.integration(yy,0.72*1.33,1.33) # перемычки 30 мкм вместо 40 мкм 1.3

In [ ]:
yy

array([  1.,   1.,   3.,   7.,  47.,  15.,  20.,  48.,  54.,   3., -11.,
        60.])

In [ ]:
len(columnsYout)

In [ ]:
pd.DataFrame(yy.astype('float'), col)

In [ ]:
yy = utils.integration(np.round(dus_mm.feedforward(np.array(XX))[:12]),nmode)

In [ ]:
a_out_train = self.feedforward(X_train)

In [ ]:
dus_mm.form_output(X_val)

In [ ]:
X_val.shape

In [ ]:
# отрисовываем результат приближения нейросетью поверх исходной функции
plt.scatter(X_val, y_val, color='black', antialiased=True)
plt.plot(X_val, dus_mm.form_output(X_val,y_val), color='magenta', linewidth=2, antialiased=True)
plt.show()

In [ ]:
f = dus_mm.gradient_checking(X,y)

In [ ]:
lambda_vec = [0]
error_train = np.zeros(len(lambda_vec))
error_val = np.zeros(len(lambda_vec))
init_theta = dus_mm.theta

for i in range(len(lambda_vec)):
  dus_mm.theta = init_theta.copy()
  dus_mm.optimize(X_train, y_train, lambda_vec[i]) 
  a_out = dus_mm.feedforward(X_train)
  a_end = a_out[:(dus_mm.sequential[-1].neurons * y_train.shape[0])]
  error_train[i] = dus_mm.compute_loss(a_end, y_train)
  
  a_out = dus_mm.feedforward(X_val)
  a_end = a_out[:(dus_mm.sequential[-1].neurons * y_val.shape[0])]
  error_val[i] = dus_mm.compute_loss(a_end, y_val)

In [ ]:
plt.plot(lambda_vec, error_train, '-o', lambda_vec, error_val, '-o', lw=2)
plt.legend(['Train', 'Cross Validation'])
plt.xlabel('lambda')
plt.ylabel('Error')

print('lambda\t\tTrain Error\tValidation Error')
for i in range(len(lambda_vec)):
    print(' %f\t%f\t%f' % (lambda_vec[i], error_train[i], error_val[i]))

In [ ]:
a_out = dus_mm.feedforward(X_val)
a_end = a_out[:(dus_mm.sequential[-1].neurons * y_val.shape[0])]

In [ ]:
a_end

In [ ]:
a_out = dus_mm.feedforward(X_val)
a_end = a_out[:(dus_mm.sequential[-1].neurons * y_val.shape[0])]
loss = dus_mm.compute_loss(a_end, y_val.ravel())

In [ ]:
loss

In [ ]:
y[1000:1001]

In [ ]:
X[1000:1001]

In [ ]:
np.round(dus_mm.feedforward(np.array(X[1000:1001]))[:12])

In [ ]:
import scipy.io
mat = scipy.io.loadmat('mnist.mat')
X = mat['X']
y = mat['y']
y[y == 10] = 0
y0 = np.zeros((y.size, 10))
for i in range(y.shape[0]):
    y0[i,y[i]] = 1

In [ ]:
dus_mm = NN_model(X.shape[1], 'loglos')
dus_mm.add(Layer(128, 'sigmoid'))
dus_mm.add(Layer(128, 'sigmoid'))
dus_mm.add(Layer(y0.shape[1], 'sigmoid'))

In [ ]:
X = np.array(X)
y = np.array(y)
lst =  np.arange(y.shape[0])
np.random.shuffle(lst)
split_data = int(0.8 * y.shape[0])

X_train = X[lst[:split_data]]
y_train = y[lst[:split_data]]
X_val = X[lst[split_data:y.shape[0]]]
y_val = y[lst[split_data:y.shape[0]]]

In [ ]:
dus_mm.optimize(X_train, y_train, X_val, y_val )

In [ ]:
dus_mm.feedforward(X[600:601,:])[:10]

In [ ]:
y0[600]

In [ ]:
 a = [1,2,3]

In [ ]:
X